In [1]:
'''
---How to use---

After make PSO by MicrogridPSO_module.py

df ,total_check, variables =loop_flowchart(PSO)
'''

import subprocess
import datetime
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import inspect

def pv_generate(PSO):
    PSO.pv=(PSO.pv_cap_max/PSO.pv_capacity_per_unit)*PSO.np_PV_efficient[PSO.h]
    return PSO.pv

def flowchart(PSO):
    
    # initialize some parameters
    PSO.pv = pv_generate(PSO)
    PSO.check = "True"
    PSO.p_diesel = 0
    PSO.trashed_power = 0
    PSO.root= str(PSO.h)+"h"
    # 太陽光の発電量が需要より多いか確認
    if PSO.pv > PSO.np_demand[PSO.h]:
        # 太陽光の発電量が需要より多い時の処理
        # バッテリーが過充電にならないかのチェック、過充電の場合、余った電気は捨てる。
        if PSO.p_battery+(PSO.pv-PSO.np_demand[PSO.h]) <= PSO.battery_max:
            PSO.p_battery = PSO.p_battery+(PSO.pv-PSO.np_demand[PSO.h])
            PSO.root=str(PSO.h)+"h: "+"Battery charge without diesel."
        else:
            PSO.trashed_power = ((PSO.pv-PSO.np_demand[PSO.h])+PSO.p_battery) - PSO.battery_max
            PSO.p_battery = PSO.battery_max
            PSO.root=str(PSO.h)+"h: "+"Fullcharge ,no diesel, trashed "\
                 + str(round(PSO.trashed_power, 2))+"[kWh]."

    else:
        # 太陽光の発電量が需要より少ない時の処理
        # ディーゼルを使うかチェック
        if PSO.p_battery > (PSO.np_demand[PSO.h]-PSO.pv) and PSO.p_battery-(PSO.np_demand[PSO.h]-PSO.pv) > PSO.battery_min:
            PSO.p_battery = PSO.p_battery-(PSO.np_demand[PSO.h]-PSO.pv)
        else:
            PSO.p_diesel = PSO.np_demand[PSO.h]-PSO.pv-(PSO.p_battery-PSO.battery_min)
            # ディーゼルの容量を超える場合はエラー
            if PSO.p_diesel > PSO.diesel_max:
                PSO.check = 'False'
                PSO.root=str(PSO.h)+"h: "+"Error! diesel capacity is over!"
            else:
                PSO.root= str(PSO.h)+"h: "+"discharging " + str(PSO.p_battery-PSO.battery_min)\
                      +"[kWh], diesel " + str( round(PSO.p_diesel, 3) ) +"[kWh]."
                PSO.p_battery = PSO.battery_min

    PSO.parameters = {"hour": PSO.h,
                  "pv": PSO.pv,
                  "battery state": PSO.p_battery,
                  "demand": PSO.np_demand[PSO.h],
                  "diesel power": PSO.p_diesel,
                  "trashed power": PSO.trashed_power,
                  "Check":PSO.check,
                  "root":PSO.root,
                    }

    return PSO.parameters, PSO.check, PSO.root

def loop_flowchart(PSO):
    #からのデータフレームを作成
    PSO.df = pd.DataFrame()
    for PSO.h in range(24):
        PSO.flowchart_parameters, PSO.check, PSO.root=flowchart(PSO)
        PSO.p_battery=PSO.flowchart_parameters['battery state']
        PSO.df=pd.concat([PSO.df,pd.io.json.json_normalize(PSO.flowchart_parameters)])

    #ループ内でfalseがあるかチェック　falseがあるとtotal_checkにfalseが入る。
    PSO.total_check = not("False" in PSO.df['Check'].values)

    PSO.variables = {"pv_cap_max": PSO.pv_cap_max,
                 "battery_cap_max": PSO.battery_cap_max,
                 "battery_max": PSO.battery_max,
                 "battery_min": PSO.battery_min,
                 "diesel_max": PSO.diesel_max,
                 "demand_sum": PSO.df['demand'].sum(),
                 "pv_power_sum": PSO.df['pv'].sum(),
                 "diesel_power_sum ": PSO.df['diesel power'].sum(),
                 "trashed_power_sum ": PSO.df['trashed power'].sum(),
                    }
    
    return PSO.df ,PSO.total_check, PSO.variables

In [2]:
#.ipynb convert to .py
#subprocess.run(['jupyter','nbconvert', '--to','python','MicrogridPSO_module_flowchart.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'MicrogridPSO_module_flowchart.ipynb'], returncode=255)